In [1]:
import pennylane as qml
import numpy as np
import torch
from torch.autograd import Variable
np.random.seed(42)

In [2]:
dev = qml.device("default.qubit", wires=4)

### Constructing  Quantum Circuit

In [33]:
def quantum_state(x):
    # initializing the quantum state with given angles of the 4 qubits
    # input x: array (4, 3)
    for i in range(nr_qubits):
        qml.RX(x[i, 0], wires=i)
        qml.RY(x[i, 1], wires=i)
        qml.RZ(x[i, 2], wires=i)

# number of qubits in the circuit
nr_qubits = 4
# number of layers in the circuit
nr_layers = 5

# randomly initialize parameters from a normal distribution
params = np.random.normal(0, np.pi, (nr_qubits, nr_layers, 3))
params = Variable(torch.tensor(params), requires_grad=True)

# a layer of the circuit ansatz
def layer(params, j):
    for i in range(nr_qubits):
        qml.RX(params[i, j, 0], wires=i)
        qml.RY(params[i, j, 1], wires=i)
        qml.RZ(params[i, j, 2], wires=i)

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[0, 3])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[1, 3])
    qml.CNOT(wires=[2, 3])
    
    
@qml.qnode(dev, interface="torch")
def circuit(params, angles):
    
    quantum_state(angles)
    
    # repeatedly apply each layer in the circuit
    for j in range(nr_layers):
        layer(params, j)

    # returns the expectation of the input matrix A on the first qubit
    return [qml.expval(qml.PauliZ(wire_n)) for wire_n in range(4)]

### Input: 4 random 4-qubit quantum states
We generate 4 randomly generated vectors of rotation angles (in radian) each of which defines a random 4-qubit quantum states.

In [4]:
x = np.random.normal(0, np.pi, (4, nr_qubits, 3))

print ("input random state")
print (x)

input random state
[[[-1.50537027 -0.58326488 -3.47565383]
  [-3.75799394  2.55262515  4.26075371]
  [-0.22622647  3.15269158  1.13611308]
  [-2.02670348  1.13535778  4.83188438]]

 [[-0.11255082  4.91547301 -8.23017197]
  [ 2.58208287  0.27346643 -0.9393593 ]
  [ 0.28827498 -6.2441319  -0.69011959]
  [ 1.12190223  4.64294107 -1.62819391]]

 [[-2.53995756 -1.57631624  2.87582057]
  [ 1.03280207 -1.66429076  1.6124772 ]
  [ 0.30497812  3.04308799 -2.20556484]
  [-1.02938099 -1.23184409 -4.59776781]]

 [[ 0.93028929  0.82012933  0.0160644 ]
  [-0.73697721 -4.44651833 -1.32149626]
  [-1.07666941 -2.52042838 -0.50669401]
  [ 1.2693632   5.92562777  0.54845237]]]


### Cost function

We want to define a cost function such that by minimizing the cost function, the optimizer will find circuit parameter that best maps the predefined input to the predefined output.

Mapping the input to state |0> is equivalent to measuring a Pauli-Z expectation value of 1, since state |0> is an eightvector of the Pauli-Z matrix with eigenvalue $\lambda = 1$. State |1> us also an eigenvector of Pauli-Z matrix with eigenvalue  $\lambda = -1$.

Here, our desired outcome for each qubit is either |0> or |1>, which is equivalent to measuring Pauli-Z value of 1 and -1 respectively. Since the Pauli-Z expecttion is bound between [-1, 1], we can define our cost function as such

$$
C = \sum_{\phi_i'=|1>}{\sigma_z\phi_i} - \sum_{\phi_i'=|0>}{\sigma_z\phi_i}
$$

where $\phi_i'$ is the target outcome of $\phi_i'$. That is, if the target outcome of a qubit is |1>, we want to minimize its Pauli-Z expectation hence adding it to the cost function, and if the target outcome of a qubit is |0>, we want to maximize its Pauli-Z expectation hence substracting it from the cost function.

In [34]:
target = [[0, 0, 1, 1],
          [0, 1, 0, 1],
          [1, 0, 1, 0],
          [1, 1, 0, 0]]

# Coefficient for cost function
# put -1 if position in target is 0, 1 if position in target is 1.
coef =   [[-1, -1, 1, 1],
          [-1, 1, -1, 1],
          [1, -1, 1, -1],
          [1, 1, -1, -1]]


def cost_fn(params):
    cost = 0
    for i in range(4):
        angles = x[i]
        out = circuit(params, angles)
        for j in range(nr_qubits):
            cost += coef[i][j] * out[j]
    return cost

Looking at the initial circuit output - the Pauli-Z expectation of each qubit, as well as the intial loss/cost without training.

In [37]:
cost = 0
for i in range(4):
    angles = x[i]
    out = circuit(params, angles)
#     print ("angle & output", angles, out)
    print ("out {}:{}".format(i, out))
    for j in range(nr_qubits):
        cost += coef[i][j] * out[j]

print ("initial cost: ", cost)

out 0:tensor([-0.0879,  0.4145,  0.2773,  0.1113], dtype=torch.float64,
       grad_fn=<CatBackward>)
out 1:tensor([ 0.1881, -0.0878, -0.4159,  0.0567], dtype=torch.float64,
       grad_fn=<CatBackward>)
out 2:tensor([0.0950, 0.1678, 0.2864, 0.1514], dtype=torch.float64,
       grad_fn=<CatBackward>)
out 3:tensor([-0.4898,  0.2625, -0.0194,  0.1909], dtype=torch.float64,
       grad_fn=<CatBackward>)
initial cost:  tensor(-0.0778, dtype=torch.float64, grad_fn=<AddBackward0>)


### Optimization
In this case, for each input 4-qubit quantum state a "perfect" circuit would have a cost/loss of -4. We sum up the loss for all 4 quantum states, so here a "perfect" loss would be -16.

In [38]:
# set up the optimizer
opt = torch.optim.Adam([params], lr=0.1)

# number of steps in the optimization routine
steps = 200

# the final stage of optimization isn't always the best, so we keep track of
# the best parameters along the way
best_cost = cost_fn(params)
best_params = np.zeros((nr_qubits, nr_layers, 3))

print("Cost after 0 steps is {:.4f}".format(cost_fn(params)))

# optimization begins
for n in range(steps):
    opt.zero_grad()
    loss = cost_fn(params)
    loss.backward()
    opt.step()

    # keeps track of best parameters
    if loss < best_cost:
        best_cost = loss
        best_params = params

    # Keep track of progress every 10 steps
    if n % 10 == 9 or n == steps - 1:
        print("Cost after {} steps is {:.4f}".format(n + 1, loss))

Cost after 0 steps is -0.0778
Cost after 10 steps is -7.1160
Cost after 20 steps is -10.6308
Cost after 30 steps is -12.0447
Cost after 40 steps is -12.6609
Cost after 50 steps is -13.0247
Cost after 60 steps is -13.2961
Cost after 70 steps is -13.3702
Cost after 80 steps is -13.4371
Cost after 90 steps is -13.5017
Cost after 100 steps is -13.6009
Cost after 110 steps is -13.7814
Cost after 120 steps is -14.0335
Cost after 130 steps is -14.2400
Cost after 140 steps is -14.4581
Cost after 150 steps is -14.5351
Cost after 160 steps is -14.5509
Cost after 170 steps is -14.5644
Cost after 180 steps is -14.5925
Cost after 190 steps is -14.6217
Cost after 200 steps is -14.6341


### Results and Output

In [29]:
def observe(zvals):
    # maps Pauli-Z expectation value to the mosly likely observed classical bit (0 or 1)
    obs = []
    for val in zvals:
        if val < 0: obs.append(1)
        else: obs.append(0)
    return obs

array_pauliz = []
for i in range(4):
    angles = x[i]
    print ("input angles")
    print (angles)
    out = circuit(params, angles)
    print ("Pauli-Z expectation: ", out.detach().numpy())
    print ("observed: ", observe(out.detach().numpy()), "\n")
    array_pauliz.append(out.detach().numpy())

input angles
[[-1.50537027 -0.58326488 -3.47565383]
 [-3.75799394  2.55262515  4.26075371]
 [-0.22622647  3.15269158  1.13611308]
 [-2.02670348  1.13535778  4.83188438]]
Pauli-Z expectation:  [ 0.95388904  0.96402192 -0.91100587 -0.9137813 ]
observed:  [0, 0, 1, 1] 

input angles
[[-0.11255082  4.91547301 -8.23017197]
 [ 2.58208287  0.27346643 -0.9393593 ]
 [ 0.28827498 -6.2441319  -0.69011959]
 [ 1.12190223  4.64294107 -1.62819391]]
Pauli-Z expectation:  [ 0.96975936 -0.98131208  0.93156533 -0.96714859]
observed:  [0, 1, 0, 1] 

input angles
[[-2.53995756 -1.57631624  2.87582057]
 [ 1.03280207 -1.66429076  1.6124772 ]
 [ 0.30497812  3.04308799 -2.20556484]
 [-1.02938099 -1.23184409 -4.59776781]]
Pauli-Z expectation:  [-0.96153854  0.82178274 -0.83231441  0.88203637]
observed:  [1, 0, 1, 0] 

input angles
[[ 0.93028929  0.82012933  0.0160644 ]
 [-0.73697721 -4.44651833 -1.32149626]
 [-1.07666941 -2.52042838 -0.50669401]
 [ 1.2693632   5.92562777  0.54845237]]
Pauli-Z expectation:  [-0.

### What happens if you provide a different state?
We test a few linear combination of our prepared initial states and use them as input. We print out the circuit output (i.e. the Pauli-Z expectation value of all the qubits), as well as the most likely observed classical bit.

In [22]:
angles = 0.1*x[0]
out = circuit(params, angles)

print ("Pauli-Z expectation of current output: ", out.detach().numpy())
print ("Observed: ", observe(out.detach().numpy()), "\n")

print ("Pauli-Z expectation of x[0] output: ", array_pauliz[0])
print ("Pauli-Z expectation of x[1] output: ", array_pauliz[1])

Pauli-Z expectation of current output:  [-0.52542141 -0.58272292  0.24538714 -0.54025831]
Observed:  [1, 1, 0, 1] 

Pauli-Z expectation of x[0] output:  [ 0.95388904  0.96402192 -0.91100587 -0.9137813 ]
Pauli-Z expectation of x[1] output:  [ 0.96975936 -0.98131208  0.93156533 -0.96714859]


In [23]:
angles = 0.5*x[0] + 0.5*x[1]
out = circuit(params, angles)

print ("Pauli-Z expectation of current output: ", out.detach().numpy())
print ("Observed: ", observe(out.detach().numpy()), "\n")

print ("Pauli-Z expectation of x[0] output: ", array_pauliz[0])
print ("Pauli-Z expectation of x[1] output: ", array_pauliz[1])

Pauli-Z expectation of current output:  [ 0.15710992 -0.32040668  0.16260218 -0.21686231]
Observed:  [0, 1, 0, 1] 

Pauli-Z expectation of x[0] output:  [ 0.95388904  0.96402192 -0.91100587 -0.9137813 ]
Pauli-Z expectation of x[1] output:  [ 0.96975936 -0.98131208  0.93156533 -0.96714859]


In [25]:
angles = 0.1*x[0] + 0.9*x[1]
out = circuit(params, angles)

print ("Pauli-Z expectation of current output: ", out.detach().numpy())
print ("Observed: ", observe(out.detach().numpy()), "\n")

print ("Pauli-Z expectation of x[0] output: ", array_pauliz[0])
print ("Pauli-Z expectation of x[1] output: ", array_pauliz[1])

Pauli-Z expectation of current output:  [ 0.44296027 -0.48294394  0.47832751 -0.44147202]
Observed:  [0, 1, 0, 1] 

Pauli-Z expectation of x[0] output:  [ 0.95388904  0.96402192 -0.91100587 -0.9137813 ]
Pauli-Z expectation of x[1] output:  [ 0.96975936 -0.98131208  0.93156533 -0.96714859]


In [27]:
angles = 0.5*x[2] + 0.5*x[3]
out = circuit(params, angles)

print ("Pauli-Z expectation of current output: ", out.detach().numpy())
print ("Observed: ", observe(out.detach().numpy()), "\n")

print ("Pauli-Z expectation of x[2] output: ", array_pauliz[2])
print ("Pauli-Z expectation of x[3] output: ", array_pauliz[3])

Pauli-Z expectation of current output:  [ 0.33370976 -0.59459027  0.04402962  0.20335267]
Observed:  [0, 1, 0, 0] 

Pauli-Z expectation of x[2] output:  [-0.96153854  0.82178274 -0.83231441  0.88203637]
Pauli-Z expectation of x[3] output:  [-0.8086572  -0.78022041  0.69692493  0.90584106]


In [28]:
angles = 0.1*x[2] + 0.9*x[3]
out = circuit(params, angles)

print ("Pauli-Z expectation of current output: ", out.detach().numpy())
print ("Observed: ", observe(out.detach().numpy()), "\n")

print ("Pauli-Z expectation of x[2] output: ", array_pauliz[2])
print ("Pauli-Z expectation of x[3] output: ", array_pauliz[3])

Pauli-Z expectation of current output:  [-0.74129708 -0.60814985  0.83410568  0.6973473 ]
Observed:  [1, 1, 0, 0] 

Pauli-Z expectation of x[2] output:  [-0.96153854  0.82178274 -0.83231441  0.88203637]
Pauli-Z expectation of x[3] output:  [-0.8086572  -0.78022041  0.69692493  0.90584106]


In general, the output can vary does not linearly interpolate. But given two very similar input state (for example, defined by rotation angle vectors that are 90%), the observed classical bit of the output will likely be the same.